In [ ]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requir

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, pipeline
from transformers import DataCollatorForTokenClassification
import torch

In [ ]:
# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the company names
company_names = ['Amazon', 'Adidas', 'Zoom', 'ICBC Turkey', 'Kale Kilit', 'Koç Holding', 'Boyner', 'Türk Telekom', 'Casio',
     'Lenovo', 'Papara', 'Migros Hemen', 'Dell', 'Teknosa', 'Havaş', 'Aksa Jeneratör', 'Pegasus Hava Yolları', 'Yataş Bedding', 'Ülker', 'TAI', 'Eti Gıda',
     'Diğer streaming servisleri', 'La Roche-Posay', 'Schwarzkopf', 'Toyota', 'Flo', 'Toshiba',
     'Clinique', 'Ege Seramik', 'Bimsa', 'HBO Max', 'Beko', 'THY', 'GittiGidiyor', 'Brisa', 'TOFAŞ', 'Efes Pilsen',
     'Mado', 'Profilo', 'Nexon', 'Diğer streaming platformları', 'MNG Kargo', 'başka uygulamalar', 'Atlasglobal',
     'Makro Market', 'Diğer müzik servisleri', 'Rowenta', 'İzocam', 'Miele', 'Asya Katılım Bankası', 'Halkbank',
     'Türkiye Sigorta', 'Sürat Kargo', 'LG', 'Breville', 'Zara', 'Buckin', 'DenizBank', 'Banvit', 'Hulu', 'Bim', 'Kenwood',
     'Samsung', 'Zorlu Enerji', 'Eucerin', 'Mavi Jeans', 'Douglas', 'Spotify', 'İş Bankası', 'İGA', 'Sahibinden',
     'Dyo Boya', 'Orka Holding', 'Baymak', 'Onur Air', 'Netflix', 'Aygaz', 'Ford',
     'Chevrolet', 'Benz', 'Avea', 'Proplan', 'YouTube', 'OpenAI', 'Sarıkamış', 'Coca-Cola', 'Microsoft',
     'Alkim Kimya', 'Kargo şirketi', 'CNR Holding', 'Roketsan', 'Tefal', 'Disney+',
     'Korkmaz', "McDonald's", 'Reeder', 'Creed', 'Pastel', 'TOGG', 'Hepsiburada', 'Sabancı Holding',
     'Evyap', 'Sanko Holding', 'IKEA', 'Lego', 'Diğer kargo şirketleri', 'Koton', 'Vodafone', 'Eti', 'Nikon', 'Trust',
     'Ziraat Bankası', 'Philips', 'Delonghi', 'Uğur Şirketler Grubu', 'Diğer müzik ugulamaları', 'Yıldız Holding',
     'Russell Hobbs', 'Tesla', 'İpek Kağıt', 'Kipa', 'Acıbadem', 'Anadolu Efes', 'Dedeman', 'Şişecam', 'DeLonghi',
     'Logitech', 'Havelsan', 'ASELSAN', 'VakıfBank', 'Yemeksepeti', 'Omsan Lojistik', 'Pegasus', 'Garanti Bankası',
     'Anadolu Isuzu', 'Opet', 'Aras Kargo', 'Bosch Rexroth', 'Çiçeksepeti', 'Namet', 'Trendyol', 'Kuveyt Türk', 'Sinbo',
     'Doğtaş', 'Diğer müzik uygulamaları',
     'Hektaş', 'BİM', 'Kahve Dünyası', 'Greyder', 'Turkcell', 'Kargo şirketleri', 'LC Waikiki', 'Migros', 'JBL',
     'Nike', 'Çaykur', 'Turkish Airlines', 'Garanti BBVA', 'Atasay', 'Starbucks', 'Ata Yatırım', 'Anadolu Jet',
     'Decathlon', 'Başka platformlar', 'Farplas', 'Hayat Kimya', 'Burger King', 'BMC', 'Petkim', 'Bioderma',
     'Assan Alüminyum', 'Albaraka Türk', 'Yapı Kredi Bankası', 'İDO', 'Halk Sigorta',
     'Sun Tekstil', 'Ekoten', 'Calvin Klein', 'Beymen', "Levi's", 'Mango',
     'Gant', 'Petrol Ofisi', 'Opet']


In [ ]:
import random
# Prepare the dataset with sentences
sentences = []
for name in company_names:
    sentences.extend([
        f"{name} çok ünlü bir firmadır.",
        f"Türkiye'nin en büyük şirketlerinden biri {name}'dir.",
        f"Kaliteli ürünleriyle tanınan {name}, sektörde liderdir.",
        f"Müşteri memnuniyeti ile ünlü {name} yenilikçi teknolojiler kullanır.",
        f"{name}, geniş ürün yelpazesi sunar ve sürdürülebilirlik konusunda çalışmaktadır.",
        f"Sosyal sorumluluk projeleri yürüten {name}, pazardaki en iyi markalardan biridir.",
        f"Ödüllü bir firma olan {name}, yüksek kalite standartlarına sahiptir.",
        f"{name}, çalışanlarına değer verir ve müşteri odaklı hizmet sunar.",
        f"{name}, sektöründe öncü bir şirkettir ve yenilikçi ürünler geliştirir.",
        f"Güçlü bir marka imajına sahip olan {name}, farklı sektörlerde faaliyet göstermektedir.",
        f"Global bir şirket olan {name}, Türkiye'nin önde gelen markalarından biridir.",
        f"Müşteri taleplerine hızlı cevap veren {name}, güvenilir hizmet sunar.",
        f"Yüksek müşteri sadakati olan {name}, geniş bir müşteri kitlesine sahiptir.",
        f"Modern üretim teknikleri kullanan {name}, çevre dostu ürünler sunar.",
        f"{name}, yüksek teknolojiye yatırım yapar ve Ar-Ge çalışmaları yapmaktadır.",
        f"{name}, geniş ürün yelpazesi sunar.",
    ])

# Add longer sentences with multiple company names
for _ in range(30):
    selected_companies = random.sample(company_names, k=random.randint(2, 6))
    sentence = " ve ".join(selected_companies) + " çok ünlü firmalardır."
    sentences.append(sentence)

    sentence = " ve ".join(selected_companies) + " sektörde liderdir ve kaliteli ürünler sunar."
    sentences.append(sentence)

    sentence = "Sosyal sorumluluk projeleri yürüten " + " ve ".join(selected_companies) + " yenilikçi teknolojiler kullanır."
    sentences.append(sentence)

    sentence = "Müşteri memnuniyeti ile ünlü " + " ve ".join(selected_companies) + " dünya çapında faaliyet göstermektedir."
    sentences.append(sentence)

# Add negative examples
negative_sentences = [
    "Bu cümlede bir şirket adı yok.",
    "Bugün hava çok güzel.",
    "Kediler ve köpekler popüler evcil hayvanlardır.",
    "Yaz tatili için nereye gitmeyi planlıyorsunuz?",
    "Python programlama dili çok popüler.",
    "Kitap okumak çok faydalıdır.",
    "Spor yapmak sağlığımız için önemlidir.",
    "Müzik dinlemek ruhu dinlendirir.",
    "Yemek yapmak bir sanattır.",
    "Film izlemek eğlencelidir.",
    "Doğa yürüyüşleri yapmak huzur vericidir.",
    "Arkadaşlarla vakit geçirmek güzeldir.",
    "Bilgisayar oyunları gençler arasında çok popülerdir.",
    "Tarihi yerleri gezmek ilginçtir.",
    "Sanat galerilerini gezmeyi seviyorum.",
    "Bahçede çalışmak rahatlatıcıdır.",
    "Yoga yapmak stresi azaltır.",
    "Uzun yürüyüşler yapmayı severim.",
    "Yeni tarifler denemek eğlencelidir.",
    "Baharda çiçekler açar.",
    "Güneşli havalarda dışarıda vakit geçirmek harikadır.",
    "Yeni yerler keşfetmek heyecan vericidir.",
    "Aile ile vakit geçirmek önemlidir.",
    "Gece gökyüzünü izlemek büyüleyicidir.",
    "Deniz kenarında yürümek huzur verir.",
    "Sıcak çikolata kış aylarında harikadır.",
    "Taze meyve yemek sağlıklıdır.",
    "Kamp yapmak doğayla iç içe olmaktır.",
    "Hızlı yemek yemek sağlıksızdır.",
    "Yeni bir dil öğrenmek faydalıdır."
]

# Combine positive and negative sentences
all_sentences = sentences + negative_sentences

In [ ]:
# Function to align labels with tokens
def align_labels_with_tokens(sentences, company_names, tokenizer):
    encodings = tokenizer(sentences, truncation=True, padding=True, return_offsets_mapping=True)
    labels = []

    for sentence, offset_mapping in zip(sentences, encodings["offset_mapping"]):
        label = [0] * len(offset_mapping)
        for company in company_names:
            start_idx = sentence.find(company)
            if start_idx != -1:
                end_idx = start_idx + len(company)
                for i, (start, end) in enumerate(offset_mapping):
                    if start >= start_idx and end <= end_idx:
                        if start == start_idx:
                            label[i] = 1  # B-COMPANY
                        else:
                            label[i] = 1  # I-COMPANY (in this case, we simplify to 1)
        labels.append(label)

    encodings.pop("offset_mapping")  # Remove offset_mapping as it is no longer needed
    encodings["labels"] = labels
    return encodings

# Tokenize and annotate the dataset
encodings = align_labels_with_tokens(all_sentences, company_names, tokenizer)

# Convert to tensors and create a custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

dataset = CustomDataset(encodings)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Increased number of epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Data collator for token classification
data_collator = DataCollatorForTokenClassification(tokenizer)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()

Step,Training Loss
10,0.000400
20,0.000300
30,0.000200
40,0.000100
50,0.000600
60,0.000100
70,0.000000
80,0.001400
90,0.000100
100,0.000100


TrainOutput(global_step=194, training_loss=0.00023490935511534227, metrics={'train_runtime': 1439.0972, 'train_samples_per_second': 2.15, 'train_steps_per_second': 0.135, 'total_flos': 66318341688144.0, 'train_loss': 0.00023490935511534227, 'epoch': 1.0})

In [ ]:

# Save the model
model.save_pretrained("./custom-ner-model-final")
tokenizer.save_pretrained("./custom-ner-model-final")


('./custom-ner-model-final/tokenizer_config.json',
 './custom-ner-model-final/special_tokens_map.json',
 './custom-ner-model-final/vocab.txt',
 './custom-ner-model-final/added_tokens.json',
 './custom-ner-model-final/tokenizer.json')

In [ ]:
# Load the model for inference
ner_model = AutoModelForTokenClassification.from_pretrained("./custom-ner-model")
ner_tokenizer = AutoTokenizer.from_pretrained("./custom-ner-model")
nlp_ner = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

# Define label map
label_map = {"LABEL_0": "O", "LABEL_1": "B-COMPANY"}

# Example usage for inference
test_sentence = "Mavi'nin kot pantolonları dayanıklılığı ile biliniyor, Kötön'un kot pantolonları ise daha hızlı yıpranıyor."
ner_results = nlp_ner(test_sentence)

# Print the results with token types
print("NER Results:")
for result in ner_results:
    token = result['word']
    entity = label_map[result['entity']]
    print(f"Token: {token}, Entity: {entity}, Score: {result['score']:.3f}")

NER Results:
Token: mavi, Entity: B-COMPANY, Score: 1.000
Token: ', Entity: O, Score: 0.998
Token: nin, Entity: O, Score: 0.996
Token: kot, Entity: O, Score: 0.997
Token: pantolon, Entity: O, Score: 0.998
Token: ##ları, Entity: O, Score: 0.999
Token: dayanıklı, Entity: O, Score: 1.000
Token: ##lı, Entity: O, Score: 1.000
Token: ##gı, Entity: O, Score: 1.000
Token: ile, Entity: O, Score: 1.000
Token: biliniyor, Entity: O, Score: 1.000
Token: ,, Entity: O, Score: 1.000
Token: kot, Entity: B-COMPANY, Score: 0.997
Token: ##on, Entity: B-COMPANY, Score: 0.918
Token: ', Entity: O, Score: 0.999
Token: un, Entity: O, Score: 0.998
Token: kot, Entity: O, Score: 0.996
Token: pantolon, Entity: O, Score: 0.997
Token: ##ları, Entity: O, Score: 0.999
Token: ise, Entity: O, Score: 0.999
Token: daha, Entity: O, Score: 0.999
Token: hızlı, Entity: O, Score: 0.999
Token: yıpran, Entity: O, Score: 0.999
Token: ##ıyor, Entity: O, Score: 0.999
Token: ., Entity: O, Score: 1.000


In [ ]:
# Function to merge subword tokens
def merge_subword_tokens(results):
    merged_results = []
    current_entity = None
    current_score = 0
    current_word = ""

    for result in results:
        if result['entity'] == 'LABEL_1':
            if current_entity is None:
                current_entity = 'B-COMPANY'
                current_word = result['word']
                current_score = result['score']
            else:
                if result['word'].startswith("##"):
                    current_word += result['word'][2:]
                else:
                    merged_results.append({
                        'word': current_word,
                        'entity': current_entity,
                        'score': current_score / len(current_word.split())
                    })
                    current_word = result['word']
                    current_score = result['score']
        else:
            if current_entity is not None:
                merged_results.append({
                    'word': current_word,
                    'entity': current_entity,
                    'score': current_score / len(current_word.split())
                })
                current_entity = None
                current_score = 0
                current_word = ""

    if current_entity is not None:
        merged_results.append({
            'word': current_word,
            'entity': current_entity,
            'score': current_score / len(current_word.split())
        })

    return merged_results

In [ ]:
# Example usage for inference
test_sentence = "Mavi'nin kot pantolonları dayanıklılığı ile biliniyor, Koton'un kot pantolonları ise daha hızlı yıpranıyor."
ner_results = nlp_ner(test_sentence)

# Merge subword tokens and filter results
merged_results = merge_subword_tokens(ner_results)

# Print the results with token types
print("NER Results:")
for result in merged_results:
    token = result['word']
    entity = result['entity']
    print(f"Token: {token}, Entity: {entity}, Score: {result['score']:.3f}")

NER Results:
Token: mavi, Entity: B-COMPANY, Score: 1.000
Token: koton, Entity: B-COMPANY, Score: 0.997


In [ ]:
import shutil
from google.colab import files

# Define the model directory and zip file name
model_dir = './custom-ner-model-final'
zip_file = 'custom-ner-model-final.zip'

# Zip the model directory
shutil.make_archive(model_dir, 'zip', model_dir)



'/content/custom-ner-model-final.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mv {zip_file} /content/drive/MyDrive/

mv: cannot stat 'custom-ner-model-final.zip': No such file or directory
